## Attention 신경망 구현 및 학습

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
pwd

'/content'

In [0]:
path = '/content/drive/My Drive/Colab Notebooks'

In [4]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 159kB/s 
     |████████████████████████████████| 92kB 12.4MB/s 
     |████████████████████████████████| 2.4MB 51.9MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0


In [0]:
import random
import tensorflow as tf
import pandas as pd
import numpy as np
from konlpy.tag import Okt
from tqdm.notebook import tqdm

## 하이퍼 파라미터

In [0]:
EPOCHS = 100
NUM_WORDS = 12958

## Encoder

In [0]:
class Encoder(tf.keras.Model):
    def __init__(self):
        super(Encoder, self).__init__()
        self.emb = tf.keras.layers.Embedding(NUM_WORDS, 64)
        self.lstm = tf.keras.layers.LSTM(512, return_sequences=True, return_state=True)

    def call(self, x, training=False, mask=None):
        x = self.emb(x)
        H, h, c = self.lstm(x)
        return H, h, c

## Decoder

In [0]:
class Decoder(tf.keras.Model):
    def __init__(self):
        super(Decoder, self).__init__()
        self.emb = tf.keras.layers.Embedding(NUM_WORDS, 64)
        self.lstm = tf.keras.layers.LSTM(512, return_sequences=True, return_state=True)
        self.att = tf.keras.layers.Attention()
        self.dense = tf.keras.layers.Dense(NUM_WORDS, activation='softmax')

    def call(self, inputs, training=False, mask=None):
        x, s0, c0, H = inputs
        x = self.emb(x)
        S, h, c = self.lstm(x, initial_state=[s0, c0])
        
        S_ = tf.concat([s0[:, tf.newaxis, :], S[:, :-1, :]], axis=1)
        A = self.att([S_, H])
        y = tf.concat([S, A], axis=-1)
        
        return self.dense(y), h, c

## Seq2seq

In [0]:
class Seq2seq(tf.keras.Model):
    def __init__(self, sos, eos):
        super(Seq2seq, self).__init__()
        self.enc = Encoder()
        self.dec = Decoder()
        self.sos = sos
        self.eos = eos

    def call(self, inputs, training=False, mask=None):
        if training is True:
            x, y = inputs
            H, h, c = self.enc(x)
            y, _, _ = self.dec((y, h, c, H))
            return y
        else:
            x = inputs
            H, h, c = self.enc(x)
            
            y = tf.convert_to_tensor(self.sos)
            y = tf.reshape(y, (1, 1))

            seq = tf.TensorArray(tf.int32, 64)

            for idx in tf.range(64):
                y, h, c = self.dec([y, h, c, H])
                y = tf.cast(tf.argmax(y, axis=-1), dtype=tf.int32)
                y = tf.reshape(y, (1, 1))
                seq = seq.write(idx, y)

                if y == self.eos:
                    break

            return tf.reshape(seq.stack(), (1, 64))

## 학습, 테스트 루프 정의

In [0]:
# Implement training loop
@tf.function
def train_step(model, inputs, labels, loss_object, optimizer, train_loss, train_accuracy):
    output_labels = labels[:, 1:]
    shifted_labels = labels[:, :-1]
    with tf.GradientTape() as tape:
        predictions = model([inputs, shifted_labels], training=True)
        loss = loss_object(output_labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)

    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(output_labels, predictions)

# Implement algorithm test
@tf.function
def test_step(model, inputs):
    return model(inputs, training=False)

## 데이터셋 준비


In [44]:
pwd

'/content'

In [0]:
dataset_file = path + '/ChatbotData.csv'
data = pd.read_csv(dataset_file, encoding='utf-8')

In [46]:
data = data.reindex(np.random.permutation(len(data)))
data

,Q,A,label
579,나가기도 귀찮아,집에서도 할 게 많아요.,0
1084,눈이 피곤해,눈 체조를 해보세요.,0
761,남친 프사에 내 사진 없어,신경쓰고 싶지 않은 사람도 있어요.,0
2653,스노클링 재미있을까?,즐거운 도전이 될 거예요.,0
6039,다시 만나서 이야기,충분한 대화 나누었길 바랍니다.,1
...,...,...,...
3594,웹툰 뭐 볼까,안 본 웹툰이 있는지 먼저 물어보고싶네요.,0
1341,동호회 약속 있어.,재미있게 놀다 오세요.,0
9598,답프로포즈 해볼까?,정말 좋아할 거예요.,2
9575,누군가를 좋아한다는건 정말 힘들일이야.,좋아하는 감정은 정말 복잡하고 힘들어요.,2


In [47]:
questions = [lines + ' \n' for lines in list(data['Q'])][:5000]
answers = ['\t ' + lines + ' \n' for lines in list(data['A'])][:5000]

print(questions)
print(answers)
num_sample = len(questions)

perm = list(range(num_sample))
random.seed(0)
random.shuffle(perm)

train_q = list()
train_a = list()
test_q = list()
test_a = list()

for idx, qna in enumerate(zip(questions, answers)):
    q, a = qna
    if perm[idx] > num_sample//5:
        train_q.append(q)
        train_a.append(a)
    else:
        test_q.append(q)
        test_a.append(a)



tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=NUM_WORDS,
                                                  filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~')

tokenizer.fit_on_texts(train_q + train_a + test_q + test_a)

train_q_seq = tokenizer.texts_to_sequences(train_q)
train_a_seq = tokenizer.texts_to_sequences(train_a)

test_q_seq = tokenizer.texts_to_sequences(test_q)
test_a_seq = tokenizer.texts_to_sequences(test_a)

x_train = tf.keras.preprocessing.sequence.pad_sequences(train_q_seq,
                                                        value=0,
                                                        padding='pre',
                                                        maxlen=64)
y_train = tf.keras.preprocessing.sequence.pad_sequences(train_a_seq,
                                                        value=0,
                                                        padding='post',
                                                        maxlen=65)


x_test = tf.keras.preprocessing.sequence.pad_sequences(test_q_seq,
                                                       value=0,
                                                       padding='pre',
                                                       maxlen=64)
y_test = tf.keras.preprocessing.sequence.pad_sequences(test_a_seq,
                                                       value=0,
                                                       padding='post',
                                                       maxlen=65)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32).prefetch(1024)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(1).prefetch(1024)


['나가기도 귀찮아 \n', '눈이 피곤해 \n', '남친 프사에 내 사진 없어 \n', '스노클링 재미있을까? \n', '다시 만나서 이야기 \n', '이러다 병걸리겠어ㅠㅠ \n', '보름밖에 안되었는데. \n', '그냥.너와 말한마디 섞고 너와의 인연에 고맙다 \n', '이별 이야기는 신중해야 하는걸 아니까 오늘도 고민합니다 \n', '사랑한다고 말을 안해줘 \n', '사랑하고 싶은데 내가 자꾸 구속해 \n', '차라리 널 만나지 않았더라면 \n', '어떡하지 \n', '가족이랑 여행 가려고 \n', '짱나는새끼 \n', '적금 들어야 하나 \n', '스테끼 먹은거 몇 년 만이냐 \n', '마음을 정리하고 있는데 뜻밖의 남자가 카톡을 했네 \n', '크루즈 타고 여행 하고 싶어 \n', '순수하게좋아했지만 걔는 너무도 약은애였어 \n', '쿠폰 다 모았다. \n', '왜 갈수록 더 생각나지 \n', '메이크업 동영상 보는 중 \n', '여자친구와 헤어진지 8일 \n', '남자친구에게 고민이나 속사정을 어디까지 얘기해? \n', '너무 힘든데 \n', '썸남이 술 취한 상태로 고백함. 기억할까? \n', '농구하다 무르팍 깨짐 \n', '드디어 어제 끝냈습니다 \n', '추우니까 나가기 싫어 \n', '이제야 오래전 상대방이 아팠던걸 이해했어. \n', '일상의 추억 \n', '연애상담하더니 둘이 사귀더라 \n', '좋아하는 사람이 너무 예뻐 \n', '여자친구가 교환학생 간대 \n', '6년째 연애중 \n', '불금입니다 \n', '거짓말은 진짜 못고치는 병 맞죠? \n', '지옥같은 5월이 다가 오네ㅜㅜ \n', '썸 타는 중인데 안 설레. \n', '먹고 자도 놀기만 해도 될까 \n', '미련이 없다고 생각했는데. \n', '부케 뭐로 하지 \n', '요즘 좋아하는 남자애가 꿈에 나오는데. \n', '만난지 얼마 안됐는데 결혼해도 될까? \n', '택배 왔나 \n', '괜히 창피해 \n', '엄마 힘들게 했어 \n', '어딘가 떠나고 싶다. \n', '

In [48]:
print(test_q)
print(test_a)

['남친 프사에 내 사진 없어 \n', '어떡하지 \n', '적금 들어야 하나 \n', '크루즈 타고 여행 하고 싶어 \n', '왜 갈수록 더 생각나지 \n', '추우니까 나가기 싫어 \n', '연애상담하더니 둘이 사귀더라 \n', '6년째 연애중 \n', '거짓말은 진짜 못고치는 병 맞죠? \n', '요즘 좋아하는 남자애가 꿈에 나오는데. \n', '어딘가 떠나고 싶다. \n', '말 거는게 어려워 \n', '비 와 \n', '기억하고 싶지 않은 기억! \n', '내일은 기다리던 소풍 간다 \n', '막장으로 가네 \n', '집들이 귀찮아 \n', '공황장애 생겼어. \n', '어젯밤에 판도라의 상자를 열어버렸네 \n', '바쁘게 사는게 나쁘지는 않네 \n', '핸드폰이 느려터졌어 \n', '남친한테 교회 가자고 하고 싶어 \n', '멋지게 나이들고 싶다 \n', '술한잔 먹고 적어 \n', '술 먹고 지갑 잃어버렸어 \n', '소개팅 앱에서 만난 사람 좋아하게 되었는데 괜찮은 사람일까요. \n', '파혼 7개월 째 \n', '사과를 안해 \n', '이별. 지쳐버렸나 \n', '스트레스 받아 \n', '혼자 살아도 될까 \n', '썸일 때도 스킨십 하고 싶어? \n', '애니 좋아하는 사람 별로야? \n', '먼저 잘게 잘자 \n', '새로운 베프가 필요해 \n', '마음이 식은 거 같아 \n', '선생님 좋아해 \n', '헤어진지 9일 \n', '이제 좀 지겨워진 거 같아 \n', '오늘도 힘들다 \n', '마음이 울적해 \n', '재미있는 드라마 끝났어 \n', '안경 쓰는 남자애 좋아하는 남자 있어? \n', '역시 나는 천재야 \n', '한 눈에 반했어. 그녀한테 좋아한다고 해도 될까. \n', '가슴 아픈 이별 \n', '아침 챙겨 먹어야지 \n', '헬스장 사람들 몸이 좋아 \n', '남자친구 스타일이 마음에 안들어. \n', '담뱃값이 왜 이렇게 비싸 \n', '장난 잼있어 \n', '좋은 아침 \n', '떠나려는 사람 잡는 방법 \n', '먹고

## 학습 환경 정의
### 모델 생성, 손실함수, 최적화 알고리즘, 평가지표 정의

In [0]:
# Create model
model = Seq2seq(sos=tokenizer.word_index['\t'],
                eos=tokenizer.word_index['\n'])

# Define loss and optimizer
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# Define performance metrics
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

## 학습 루프 동작

In [50]:
for epoch in tqdm(range(EPOCHS)):
    for seqs, labels in train_ds:
        train_step(model, seqs, labels, loss_object, optimizer, train_loss, train_accuracy)

    template = 'Epoch {}, Loss: {}, Accuracy: {}'
    print(template.format(epoch + 1,
                          train_loss.result(),
                          train_accuracy.result() * 100))

    train_loss.reset_states()
    train_accuracy.reset_states()

Epoch 1, Loss: 0.9679239392280579, Accuracy: 92.15116882324219
Epoch 2, Loss: 0.5164560079574585, Accuracy: 93.98716735839844
Epoch 3, Loss: 0.47447463870048523, Accuracy: 94.2559814453125
Epoch 4, Loss: 0.4455713629722595, Accuracy: 94.3380355834961
Epoch 5, Loss: 0.4288519024848938, Accuracy: 94.40953826904297
Epoch 6, Loss: 0.41652077436447144, Accuracy: 94.44197082519531
Epoch 7, Loss: 0.4050038456916809, Accuracy: 94.47830963134766
Epoch 8, Loss: 0.39389508962631226, Accuracy: 94.51737976074219
Epoch 9, Loss: 0.3824133574962616, Accuracy: 94.55371856689453
Epoch 10, Loss: 0.37083354592323303, Accuracy: 94.60450744628906
Epoch 11, Loss: 0.3589758574962616, Accuracy: 94.66859436035156
Epoch 12, Loss: 0.3465612232685089, Accuracy: 94.7154769897461
Epoch 13, Loss: 0.3334851562976837, Accuracy: 94.80924987792969
Epoch 14, Loss: 0.3195759952068329, Accuracy: 94.90849304199219
Epoch 15, Loss: 0.3049016296863556, Accuracy: 95.0249252319336
Epoch 16, Loss: 0.2895081639289856, Accuracy: 95.

## 테스트 루프

In [51]:
for test_seq, test_labels in test_ds:
    prediction = test_step(model, test_seq)
    test_text = tokenizer.sequences_to_texts(test_seq.numpy())
    gt_text = tokenizer.sequences_to_texts(test_labels.numpy())
    texts = tokenizer.sequences_to_texts(prediction.numpy())
    print('_')
    print('질문: ', test_text)
    print('정답: ', gt_text)
    print('예상답: ', texts)

_
질문:  ['남친 프사에 내 사진 없어 \n']
정답:  ['\t 신경쓰고 싶지 않은 사람도 있어요 \n']
예상답:  ['갑자기 말하면 상대방이 당황하지 않을까요 \n']
_
질문:  ['어떡하지 \n']
정답:  ['\t 어떡하면 좋을까요 \n']
예상답:  ['성공하길 바랄게요 \n']
_
질문:  ['적금 들어야 하나 \n']
정답:  ['\t 적금 들 수 있으면 드는 게 좋죠 \n']
예상답:  ['귀찮아서 인정부터 시작이죠 \n']
_
질문:  ['크루즈 타고 여행 하고 싶어 \n']
정답:  ['\t 멋진 생각이네요 \n']
예상답:  ['사랑하는데 부담스럽지 만들어보세요 \n']
_
질문:  ['왜 갈수록 더 생각나지 \n']
정답:  ['\t 너무 많은 생각을 하지 마세요 \n']
예상답:  ['그러니까 싸우는 거예요 \n']
_
질문:  ['추우니까 나가기 싫어 \n']
정답:  ['\t 겨울에는 귤 먹으면서 집에 있는게 최고죠 \n']
예상답:  ['생각들을 정리하게 좋겠네요 \n']
_
질문:  ['연애상담하더니 둘이 사귀더라 \n']
정답:  ['\t 대화를 하다가 친해졌나봐요 \n']
예상답:  ['얼른 실내로 들어가세요 \n']
_
질문:  ['6년째 연애중 \n']
정답:  ['\t 부럽네요 \n']
예상답:  ['차분하게 정리할 시간이 필요해요 \n']
_
질문:  ['거짓말은 진짜 못고치는 병 맞죠 \n']
정답:  ['\t 아무래도 그렇죠 \n']
예상답:  ['얼른 멋을 내보세요 \n']
_
질문:  ['요즘 좋아하는 남자애가 꿈에 나오는데 \n']
정답:  ['\t 생각을 많이 해서 그런가봐요 \n']
예상답:  ['안전 사람도 필요할 거예요 \n']
_
질문:  ['어딘가 떠나고 싶다 \n']
정답:  ['\t 저도요 \n']
예상답:  ['주말에 gogo \n']
_
질문:  ['말 거는게 어려워 \n']
정답:  ['\t 처음만 어렵지 그 다음부터는 덜할 거예요 \n']
예상답:  ['말로 하는 상처는 지울 수도 

In [0]:
word_zip = list()
for idx, qna in enumerate(zip(questions, answers)):
  
  x,y = qna
  word_zip += x.split(' ')
  word_zip += y.split(' ')


In [37]:
ex_list = list(set(word_zip))
len(ex_list)

12958